In [2]:

# 경고창 무시
import warnings
warnings.filterwarnings('ignore')

## 가설 : 전체 키워드 대비 각각의 키워드 차지 비율과 전국 관광지 방문객 대비 해당 지역 방문객 비율의 월별 변화량이 상관관계가 있다. 

## 데이터 불러오기

In [3]:
import pandas as pd
import pymongo
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
mpl.rc('font', family='NanumBarunGothic') # 혹은 다른 설치한 Nanum 폰트 사용
import scipy
import scipy.stats as stats

client=pymongo.MongoClient("192.168.10.240",27017)
db= client['AI_LKJ']
collection_attraction = db['data_attraction']
collection_concept = db['data_concept_search']
collection_trend = db['data_trend_search']
data_attraction = collection_attraction.find()
data_concept = collection_concept.find()
data_trend = collection_trend.find()
df_attraction = pd.DataFrame(data_attraction)
df_concept = pd.DataFrame(data_concept)
df_visitor = df_concept.query("destination_type =='전체'")
df_concept=df_concept.query("destination_type !='전체'")
df_trend = pd.DataFrame(data_trend)

In [4]:
df_trend

,_id,region,std_year_month,std_year,std_month,tour_trend,num_mention
0,65e92ddfbb908e26c804393d,강원,202007,2020,7,레포츠,58868
1,65e92ddfbb908e26c804393e,강원,202008,2020,8,레포츠,53624
2,65e92ddfbb908e26c804393f,강원,202006,2020,6,레포츠,49214
3,65e92ddfbb908e26c8043940,강원,202010,2020,10,레포츠,47223
4,65e92ddfbb908e26c8043941,강원,202007,2020,7,휴식/힐링,40740
...,...,...,...,...,...,...,...
4075,65e92de0bb908e26c8044928,충북,202301,2023,1,체험,868
4076,65e92de0bb908e26c8044929,충북,202302,2023,2,미식,843
4077,65e92de0bb908e26c804492a,충북,202312,2023,12,미식,832
4078,65e92de0bb908e26c804492b,충북,202311,2023,11,미식,797


In [5]:
df_concept

,_id,destination_type,destination_search,region,std_year,std_month,std_year_month
12,65e92d9cbb908e26c804032a,숙박,440809,강원,2020,1,202001
13,65e92d9cbb908e26c804032b,음식,558197,강원,2020,1,202001
14,65e92d9cbb908e26c804032c,기타관광,198533,강원,2020,1,202001
15,65e92d9cbb908e26c804032d,쇼핑,121124,강원,2020,1,202001
16,65e92d9cbb908e26c804032e,문화관광,197814,강원,2020,1,202001
...,...,...,...,...,...,...,...
8155,65e92d9dbb908e26c80422f9,문화관광,80761,충북,2023,12,202312
8156,65e92d9dbb908e26c80422fa,역사관광,20072,충북,2023,12,202312
8157,65e92d9dbb908e26c80422fb,자연관광,25920,충북,2023,12,202312
8158,65e92d9dbb908e26c80422fc,체험관광,11141,충북,2023,12,202312


In [6]:
df_visitor

,_id,destination_type,destination_search,region,std_year,std_month,std_year_month
0,65e92d9cbb908e26c804031e,전체,1937279,강원,2020,1,202001
1,65e92d9cbb908e26c804031f,전체,1395025,강원,2020,2,202002
2,65e92d9cbb908e26c8040320,전체,1295467,강원,2020,3,202003
3,65e92d9cbb908e26c8040321,전체,1592147,강원,2020,4,202004
4,65e92d9cbb908e26c8040322,전체,2286676,강원,2020,5,202005
...,...,...,...,...,...,...,...
8047,65e92d9dbb908e26c804228d,전체,853451,충북,2023,1,202301
8048,65e92d9dbb908e26c804228e,전체,954844,충북,2023,2,202302
8049,65e92d9dbb908e26c804228f,전체,1677621,충북,2023,10,202310
8050,65e92d9dbb908e26c8042290,전체,1272051,충북,2023,11,202311


In [7]:
df_concept['destination_type'].unique()

array(['숙박', '음식', '기타관광', '쇼핑', '문화관광', '역사관광', '자연관광', '체험관광', '레저스포츠'],
      dtype=object)

In [8]:
df_attraction

,_id,rank,attraction_name,address,classification,attraction_search,region,std_year
0,65e91282bb908e26c8038353,1,속초관광수산시장,강원 속초시 중앙로147번길 16-0,시장,644655,강원,2020
1,65e91282bb908e26c8038354,2,속초해변,강원 속초시 해오름로 186-0,자연경관(하천/해양),276727,강원,2020
2,65e91282bb908e26c8038355,3,경포해변,강원 강릉시,자연경관(하천/해양),263395,강원,2020
3,65e91282bb908e26c8038356,4,주문진항,강원 강릉시 해안로 1758-14,자연경관(하천/해양),215243,강원,2020
4,65e91282bb908e26c8038357,5,쏠비치삼척,강원 삼척시 수로부인길 453-0,콘도미니엄,204957,강원,2020
...,...,...,...,...,...,...,...,...
13595,65e91283bb908e26c803b86e,96,듀레베이커리호암점,충북 충주시 중원대로 3250-0,간이음식,5596,충북,2023
13596,65e91283bb908e26c803b86f,97,오시며가시며,충북 충주시 중방곡길 56-0,한식,5591,충북,2023
13597,65e91283bb908e26c803b870,98,삼학도수산,충북 청주시 흥덕구 비하로 30-0,전문음식,5538,충북,2023
13598,65e91283bb908e26c803b871,99,청주본가청원직영점,충북 청주시 서원구 청남로 889-0,한식,5506,충북,2023


## 데이터 전처리

### attraction 분류

In [9]:
df_attraction['classification'].unique()

array(['시장', '자연경관(하천/해양)', '콘도미니엄', '종교성지', '기타관광', '호텔', '수상레저스포츠',
       '교통시설', '복합관광시설', '기타문화관광지', '육상레저스포츠', '랜드마크관광', '테마공원', '역사유적지',
       '자연공원', '기타레저스포츠', '전시시설', '농/산/어촌체험', '자연경관(산)', '쇼핑몰', '백화점',
       '도시공원', '공연시설', '대형마트', '웰니스관광', '캠핑', '레저스포츠시설', '모텔', '기타쇼핑시설',
       '기타숙박', '데이트코스', '펜션/민박', '역사유물', '자연생태', '자연관광(산)', '면세점', '한식',
       '음식점기타', '카페/찻집', '전문음식', '간이음식', '외국식'], dtype=object)

In [10]:
df_attraction= df_attraction.drop(df_attraction.query("classification =='교통시설'").index)

In [11]:
df_attraction.loc[df_attraction.query("classification in ['카페/찻집', '간이음식', '한식', '전문음식', '외국식','음식점기타']").index,'destination_type'] = '음식'
# ['카페/찻집', '간이음식', '한식', '전문음식', '외국식', '음식점기타']

In [12]:
df_attraction.loc[df_attraction.query("classification == ['도시공원', '자연경관(하천/해양)','자연경관(산)','자연관광(산)', '자연생태','데이트코스', '자연공원','랜드마크관광']").index,'destination_type']='자연관광'

In [13]:
df_attraction.loc[df_attraction.query("classification == ['쇼핑몰', '백화점', '대형마트','면세점']").index,'destination_type']='쇼핑'

In [14]:
df_attraction.loc[df_attraction.query("classification == ['시장', '전시시설', '공연시설', '기타문화관광지', '테마공원','웰니스관광','농/산/어촌체험','역사유적지','종교성지']").index,'destination_type']='문화관광'

In [15]:
df_attraction.loc[df_attraction.query("classification == ['육상레저스포츠','레저스포츠시설','캠핑','수상레저스포츠', '기타레저스포츠']").index,'destination_type']='레저스포츠'

In [16]:
df_attraction.loc[df_attraction.query("classification == ['콘도미니엄', '호텔', '기타숙박']").index,'destination_type']='숙박'

In [17]:
df_attraction['destination_type'].unique()

array(['문화관광', '자연관광', '숙박', nan, '레저스포츠', '쇼핑', '음식'], dtype=object)

In [18]:
set(df_attraction.loc[df_attraction['destination_type'].isna(),'attraction_name'].to_list())

{'J호텔',
 '간월재',
 '간절곶',
 '간절곶소망우체통',
 '감천문화마을',
 '갓바위',
 '강원랜드',
 '갤러리모텔',
 '거창Y자형출렁다리',
 '경북천년숲정원',
 '경주버드파크',
 '경주양남주상절리',
 '경주화랑의언덕',
 '고석정국민관광지',
 '공구의거리',
 '공업탑',
 '광주김치타운',
 '광한루원',
 '구시청사거리',
 '국립생태원',
 '국립세종수목원연구동',
 '그랜드모텔',
 '금강보행교',
 '금강하구둑',
 '금호모텔',
 '기장해녀촌',
 '남원큰엉',
 '남해상상양떼목장편백숲',
 '내장산케이블카',
 '뉴세종모텔',
 '당산파크',
 '대관령양떼목장',
 '대둔산케이블카',
 '대왕암공원출렁다리',
 '대전신세계Art&Science',
 '도담삼봉',
 '도도리파크',
 '독일마을',
 '동의보감촌',
 '동촌유원지',
 '동피랑벽화마을',
 '두물머리',
 '등기산스카이워크',
 '등억온천지구',
 '레지던스호텔라인',
 '로얄모텔',
 '마장호수출렁다리',
 '머루와인동굴',
 '메타세쿼이아가로수길',
 '메타프로방스',
 '목포해상케이블카북항승강장',
 '무한의다리',
 '뮤즈모텔',
 '발렌타인모텔',
 '발왕산케이블카',
 '방축천음악분수',
 '백제문화단지',
 '베스트인시티호텔',
 '벽골제',
 '보롬왓',
 '보문관광단지',
 '보성녹차밭',
 '부소담악',
 '브라운도트호텔첨단점',
 '사인암',
 '사천바다케이블카대방정류장',
 '산굼부리',
 '산막이옛길',
 '삼성궁',
 '삼양대관령목장',
 '삼양라운드힐',
 '삽교호관광지',
 '샤인모텔',
 '선녀바위',
 '선바위',
 '설리스카이워크',
 '설악케이블카',
 '섬진강기차마을',
 '섭지코지',
 '세븐모텔',
 '세븐브릭스호텔',
 '세종어린이천문대',
 '세종필무인텔',
 '세천유원지',
 '속초아이대관람차',
 '송도해상케이블카',
 '송산유원지',
 '쇠소깍',
 '수성유원지',
 '수양개빛터널',
 '순창용궐산하늘길',


In [19]:
df_attraction.loc[df_attraction.loc[df_attraction['attraction_name'].isin(['정동진','송산유원지','운문댐하류보유원지','송산유원지','지산유원지','동촌유원지','수성유원지','세천유원지','산굼부리','벽골제','간월재','간절곶','간절곶소망우체통','갓바위','거창Y자형출렁다리','경북천년숲정원','경주양남주상절리','경주화랑의언덕','남원큰엉','내장산케이블카','대둔산케이블카','대왕암공원출렁다리','도도리파크','두물머리','등기산 스카이워크','마장호수 출렁다리','머루와인 동굴','보성 녹차밭','보롬왓','부소담악','발왕산 케이블카','목포해상케이블카 북항승강장','사인암','사천바다케이블카대방정류장','산막이옛길','삼성궁','삼양라운드힐','선녀바위','선바위','설악케이블카','섭지코지','송도해상케이블카','쇠소깍','순창용궐산하늘길','순천만습지','신비의도로','안반데기','앞산케이블카','여수해상케이블카놀아정류장','여수해상케이블카돌산정류장','영남알프스얼음골케이블카','예당호출렁다리','옐로우출렁다리','옥정호출렁다리','용두암','용머리해안','원주소금산출렁다리','월영교','월영산출렁다리','마장호수출렁다리','이응다리','머루와인동굴','메타세쿼이아가로수길','보성녹차밭','발왕산케이블카','목포해상케이블카북항승강장','임진각','자드락숲','정동원길','정서진','제천옥순봉출렁다리','주상절리대','채계산출렁다리','채석강','천장호출렁다리','철원한탄강은하수교','청남대','청자다리','청풍호반케이블카','촛대바위','춘천삼악산호수케이블카','탑정호출렁다리','태안신두리해안사구','통영케이블카','팔공산케이블카','퍼플교','현충사곡교천은행나무길','황령산봉수대'])].index,"destination_type"] = '자연관광'

In [20]:
df_attraction.loc[df_attraction.loc[df_attraction['attraction_name'].isin(['고석정국민관광지','보문관광단지','등억온천지구','메타프로방스','중문관광단지','삽교호관광지','청풍문화재단지','하슬라아트월드','강원랜드','백제문화단지','경주버드파크','파라다이스시티원더박스','감천문화마을','공구의거리','공업탑','광주김치타운','광한루원','구시청사거리','국립생태원','국립세종수목원연구동','금강보행교','금강하구둑','기장해녀촌','남해상상양떼목장편백숲','대관령양떼목장','독일마을','동의보감촌','동피랑벽화마을','방축천 음악분수','메타세쿼이아 가로수길','삼양대관령목장','설리스카이워크','섬진강기차마을','세종어린이천문대','수양개빛터널','순천만국가정원','순천오픈세트장','스누피가든','엑스포한빛탑','영산강6경(승촌보)승촌보문화관','옥토끼우주센터','와인터널','장생포고래문화마을','지중해마을','태권도원','펭귄마을','방축천음악분수','저도연육교스카이워크','해상스카이워크','등기산스카이워크','해운대블루라인파크미포정거장','무한의다리','헤이리예술마을','헬로키티아일랜드','흰여울문화마을'])].index,"destination_type"] = '문화관광'

In [21]:
df_attraction.loc[df_attraction.loc[df_attraction['attraction_name'].isin(['휘닉스평창','레지던스호텔라인','브라운도트호텔첨단점','아난티코브','제주신화월드','천안상록리조트','J호텔','갤러리모텔','그랜드모텔','금호모텔','뉴세종모텔','당산파크','도담삼봉','레지던스 호텔라인','로얄모텔','뮤즈모텔','발렌타인모텔','브라운도트호텔 첨단점','베스트인시티호텔','샤인모텔','세븐모텔','세븐브릭스호텔','세종필무인텔','스카이무인호텔','시크릿','신라모텔','자자무인텔','청벽비발디펜션','커플링모텔','프로방스','필사랑무인텔','하루모텔','호텔더테라스','호텔세븐','휴휴당한옥펜션',
'힐링빌리지펜션'])].index,"destination_type"] = '숙박'

In [22]:
df_attraction.loc[df_attraction.loc[df_attraction['attraction_name'].isin(['대전신세계Art&Science',])].index,"destination_type"] = '쇼핑'

In [23]:
df_attraction.dropna(subset=['destination_type'], inplace=True)

In [24]:
df_attraction.isnull().sum()

_id                  0
rank                 0
attraction_name      0
address              0
classification       0
attraction_search    0
region               0
std_year             0
destination_type     0
dtype: int64

In [25]:
df_attraction['destination_type'].unique()

array(['문화관광', '자연관광', '숙박', '레저스포츠', '쇼핑', '음식'], dtype=object)

In [26]:
conditiona = "region =='충북'"
conditionb = "destination_type =='레저스포츠'"
conditions = f"{conditiona} and {conditionb}"
df_attraction.query(conditions)

,_id,rank,attraction_name,address,classification,attraction_search,region,std_year,destination_type
1610,65e91282bb908e26c803899d,11,떼제베CC,충북 청주시 흥덕구 동림2길 149-0,육상레저스포츠,65630,충북,2020,레저스포츠
1617,65e91282bb908e26c80389a4,18,대영힐스CC,충북 충주시 성종두담길 114-0,육상레저스포츠,49514,충북,2020,레저스포츠
1620,65e91282bb908e26c80389a7,21,진양밸리골프클럽,충북 음성군 금일로 1195-0,육상레저스포츠,40738,충북,2020,레저스포츠
1622,65e91282bb908e26c80389a9,23,히든밸리GC,충북 진천군 소토골길 61-0,육상레저스포츠,40214,충북,2020,레저스포츠
1623,65e91282bb908e26c80389aa,24,임페리얼레이크CC,충북 충주시 다래울길 52-0,육상레저스포츠,38591,충북,2020,레저스포츠
...,...,...,...,...,...,...,...,...,...
11871,65e91283bb908e26c803b1b2,72,이븐데일CC,충북 청주시 상당구 대신2길 31-0,육상레저스포츠,20960,충북,2023,레저스포츠
11874,65e91283bb908e26c803b1b5,75,세일CC,충북 충주시 동락길 207-0,육상레저스포츠,20274,충북,2023,레저스포츠
11878,65e91283bb908e26c803b1b9,79,일레븐CC,충북 충주시 북부로 980-0,육상레저스포츠,19623,충북,2023,레저스포츠
11885,65e91283bb908e26c803b1c0,86,청풍호관광모노레일,충북 제천시 청풍명월로 879-17,기타레저스포츠,17955,충북,2023,레저스포츠


In [27]:
list_data = []
for x in list(df_attraction['region'].unique()):
    for y in list(df_attraction['destination_type'].unique()):
        data_dict = {}
        data_dict['region']=x
        data_dict['destination_type']=y
        data_dict['attraction_search']=df_attraction.loc[(df_attraction['region']==x) & (df_attraction['destination_type']==y),'attraction_search'].sum()
        list_data.append(data_dict)
data_attraction_sum = pd.DataFrame(data=list_data)
data_attraction_sum

,region,destination_type,attraction_search
0,강원,문화관광,9738098
1,강원,자연관광,10257543
2,강원,숙박,9301901
3,강원,레저스포츠,2744666
4,강원,쇼핑,0
...,...,...,...
97,충북,자연관광,2985789
98,충북,숙박,1377916
99,충북,레저스포츠,4896847
100,충북,쇼핑,927965


### concept 분류

In [28]:
df_concept['destination_type'].unique()

array(['숙박', '음식', '기타관광', '쇼핑', '문화관광', '역사관광', '자연관광', '체험관광', '레저스포츠'],
      dtype=object)

In [29]:
df_concept=df_concept.query("destination_type != ['전체','기타관광']")
df_concept['destination_type'].unique()

array(['숙박', '음식', '쇼핑', '문화관광', '역사관광', '자연관광', '체험관광', '레저스포츠'],
      dtype=object)

In [30]:
df_concept.loc[df_concept.query("destination_type == ['체험관광','역사관광']").index,'destination_type']='문화관광'

In [31]:
df_concept['destination_type'].unique()

array(['숙박', '음식', '쇼핑', '문화관광', '자연관광', '레저스포츠'], dtype=object)

In [32]:
list_data = []
for x in list(df_concept['std_month'].unique()):
    for y in list(df_concept['destination_type'].unique()):
        data_dict = {}
        data_dict['std_month']=x
        data_dict['destination_type']=y
        data_dict['destination_search']=df_concept.loc[(df_concept['std_month']==x) & (df_concept['destination_type']==y),'destination_search'].sum()
        list_data.append(data_dict)
data_concept_sum = pd.DataFrame(data=list_data)
data_concept_sum

,std_month,destination_type,destination_search
0,1,숙박,10695561
1,1,음식,39818247
2,1,쇼핑,17019729
3,1,문화관광,14624554
4,1,자연관광,5333026
...,...,...,...
67,12,음식,51746719
68,12,쇼핑,20139183
69,12,문화관광,16264725
70,12,자연관광,5184728


### visitor 분류

In [33]:
df_visitor

,_id,destination_type,destination_search,region,std_year,std_month,std_year_month
0,65e92d9cbb908e26c804031e,전체,1937279,강원,2020,1,202001
1,65e92d9cbb908e26c804031f,전체,1395025,강원,2020,2,202002
2,65e92d9cbb908e26c8040320,전체,1295467,강원,2020,3,202003
3,65e92d9cbb908e26c8040321,전체,1592147,강원,2020,4,202004
4,65e92d9cbb908e26c8040322,전체,2286676,강원,2020,5,202005
...,...,...,...,...,...,...,...
8047,65e92d9dbb908e26c804228d,전체,853451,충북,2023,1,202301
8048,65e92d9dbb908e26c804228e,전체,954844,충북,2023,2,202302
8049,65e92d9dbb908e26c804228f,전체,1677621,충북,2023,10,202310
8050,65e92d9dbb908e26c8042290,전체,1272051,충북,2023,11,202311


In [34]:
list_data = []
for x in list(df_visitor['region'].unique()):
    for y in list(df_visitor['std_month'].unique()):
        data_dict = {}
        data_dict['region']=x
        data_dict['std_month']=y
        data_dict['destination_search']=df_visitor.loc[(df_visitor['region']==x) & (df_visitor['std_month']==y),'destination_search'].sum()
        list_data.append(data_dict)
data_visitor_sum = pd.DataFrame(data=list_data)
data_visitor_sum

,region,std_month,destination_search
0,강원,1,7767478
1,강원,2,8184410
2,강원,3,6266780
3,강원,4,8407164
4,강원,5,10827689
...,...,...,...
199,충북,8,5076644
200,충북,9,4566436
201,충북,10,5873293
202,충북,11,4562010


## DDA & EDA

### 관광지 유형별 방문자가 많은 지역
* 레저스포츠 : 충북, 경기, 강원, 인천, 경남
* 문화관광 : 경기, 서울, 강원, 경북, 인천
* 쇼핑 : 경기, 서울, 부산, 인천, 대전
* 숙박 : 강원, 서울, 제주, 부산, 인천
* 음식 : 경기, 강원, 서울, 인천, 제주
* 자연관광 : 경기, 강원, 인천, 제주, 충남

In [35]:
pivot_attraction=pd.pivot_table(data=df_attraction,index="destination_type",columns = "region",values='attraction_search',aggfunc='sum')
pivot_attraction

region,강원,경기,경남,경북,광주,대구,대전,부산,서울,세종,울산,인천,전남,전북,제주,충남,충북
destination_type,,,,,,,,,,,,,,,,,
레저스포츠,2744666.0,2890460.0,2146843.0,1879754.0,349171.0,489501.0,640613.0,1120255.0,818317.0,552045.0,615157.0,2362901.0,1161617.0,1372462.0,736423.0,1485952.0,4896847.0
문화관광,9738098.0,14317403.0,6600853.0,7426833.0,2136170.0,3899587.0,1917585.0,6225144.0,13789047.0,1336886.0,1468137.0,6631686.0,3603443.0,3730702.0,4591529.0,6454280.0,1972155.0
쇼핑,NaN,48918389.0,2392445.0,173009.0,1732516.0,3930381.0,4037354.0,9010754.0,29481866.0,96362.0,1104535.0,5789551.0,498887.0,573571.0,148212.0,1819608.0,927965.0
숙박,9301901.0,866923.0,1348966.0,1870344.0,508094.0,1034349.0,994710.0,2966147.0,7219625.0,264232.0,496528.0,2576608.0,1869836.0,1209553.0,4091583.0,1397756.0,1377916.0
음식,10524341.0,15162870.0,4108589.0,5065397.0,2276342.0,3285110.0,3740999.0,5844877.0,8221779.0,2654369.0,2211646.0,7597506.0,4959672.0,3926896.0,7072096.0,5031953.0,3526012.0
자연관광,10257543.0,10475938.0,2668337.0,3621794.0,441888.0,1990408.0,1241020.0,5981596.0,6606251.0,853113.0,2510582.0,9970233.0,4099961.0,2787978.0,8530982.0,6924783.0,2985789.0


In [36]:
for i in list(pivot_attraction.index):
               print("* {} : {}, {}, {}, {}, {}".format(i,pivot_attraction.loc[i].sort_values(ascending=False).keys()[0],pivot_attraction.loc[i].sort_values(ascending=False).keys()[1],pivot_attraction.loc[i].sort_values(ascending=False).keys()[2],pivot_attraction.loc[i].sort_values(ascending=False).keys()[3],pivot_attraction.loc[i].sort_values(ascending=False).keys()[4]))

* 레저스포츠 : 충북, 경기, 강원, 인천, 경남
* 문화관광 : 경기, 서울, 강원, 경북, 인천
* 쇼핑 : 경기, 서울, 부산, 인천, 대전
* 숙박 : 강원, 서울, 제주, 부산, 인천
* 음식 : 경기, 강원, 서울, 인천, 제주
* 자연관광 : 경기, 강원, 인천, 제주, 충남


### 지역별 방문자 수가 많은 관광지 유형
* 강원 : 음식, 자연관광, 문화관광, 숙박, 레저스포츠
* 경기 : 쇼핑, 음식, 문화관광, 자연관광, 레저스포츠
* 경남 : 문화관광, 음식, 자연관광, 쇼핑, 레저스포츠
* 경북 : 문화관광, 음식, 자연관광, 레저스포츠, 숙박
* 광주 : 음식, 문화관광, 쇼핑, 숙박, 자연관광
* 대구 : 쇼핑, 문화관광, 음식, 자연관광, 숙박
* 대전 : 쇼핑, 음식, 문화관광, 자연관광, 숙박
* 부산 : 쇼핑, 문화관광, 자연관광, 음식, 숙박
* 서울 : 쇼핑, 문화관광, 음식, 숙박, 자연관광
* 세종 : 음식, 문화관광, 자연관광, 레저스포츠, 숙박
* 울산 : 자연관광, 음식, 문화관광, 쇼핑, 레저스포츠
* 인천 : 자연관광, 음식, 문화관광, 쇼핑, 숙박
* 전남 : 음식, 자연관광, 문화관광, 숙박, 레저스포츠
* 전북 : 음식, 문화관광, 자연관광, 레저스포츠, 숙박
* 제주 : 자연관광, 음식, 문화관광, 숙박, 레저스포츠
* 충남 : 자연관광, 문화관광, 음식, 쇼핑, 레저스포츠
* 충북 : 레저스포츠, 음식, 자연관광, 문화관광, 숙박

In [37]:
pivot_attraction=pd.pivot_table(data=df_attraction,index="region",columns = "destination_type",values='attraction_search',aggfunc='sum')
pivot_attraction

destination_type,레저스포츠,문화관광,쇼핑,숙박,음식,자연관광
region,,,,,,
강원,2744666.0,9738098.0,NaN,9301901.0,10524341.0,10257543.0
경기,2890460.0,14317403.0,48918389.0,866923.0,15162870.0,10475938.0
경남,2146843.0,6600853.0,2392445.0,1348966.0,4108589.0,2668337.0
경북,1879754.0,7426833.0,173009.0,1870344.0,5065397.0,3621794.0
광주,349171.0,2136170.0,1732516.0,508094.0,2276342.0,441888.0
대구,489501.0,3899587.0,3930381.0,1034349.0,3285110.0,1990408.0
대전,640613.0,1917585.0,4037354.0,994710.0,3740999.0,1241020.0
부산,1120255.0,6225144.0,9010754.0,2966147.0,5844877.0,5981596.0
서울,818317.0,13789047.0,29481866.0,7219625.0,8221779.0,6606251.0


In [38]:
list_concept_region = {'레저스포츠':[],'문화관광':[],'쇼핑':[],'숙박':[],'음식':[],'자연관광':[]}
for i in list(pivot_attraction.index):
    print("* {} : {}, {}, {}, {}, {}".format(i,pivot_attraction.loc[i].sort_values(ascending=False).keys()[0],pivot_attraction.loc[i].sort_values(ascending=False).keys()[1],pivot_attraction.loc[i].sort_values(ascending=False).keys()[2],pivot_attraction.loc[i].sort_values(ascending=False).keys()[3],pivot_attraction.loc[i].sort_values(ascending=False).keys()[4]))
    list_concept_region[pivot_attraction.loc[i].sort_values(ascending=False).keys()[0]].append(i)
    list_concept_region[pivot_attraction.loc[i].sort_values(ascending=False).keys()[1]].append(i)
    list_concept_region[pivot_attraction.loc[i].sort_values(ascending=False).keys()[2]].append(i)

* 강원 : 음식, 자연관광, 문화관광, 숙박, 레저스포츠
* 경기 : 쇼핑, 음식, 문화관광, 자연관광, 레저스포츠
* 경남 : 문화관광, 음식, 자연관광, 쇼핑, 레저스포츠
* 경북 : 문화관광, 음식, 자연관광, 레저스포츠, 숙박
* 광주 : 음식, 문화관광, 쇼핑, 숙박, 자연관광
* 대구 : 쇼핑, 문화관광, 음식, 자연관광, 숙박
* 대전 : 쇼핑, 음식, 문화관광, 자연관광, 숙박
* 부산 : 쇼핑, 문화관광, 자연관광, 음식, 숙박
* 서울 : 쇼핑, 문화관광, 음식, 숙박, 자연관광
* 세종 : 음식, 문화관광, 자연관광, 레저스포츠, 숙박
* 울산 : 자연관광, 음식, 문화관광, 쇼핑, 레저스포츠
* 인천 : 자연관광, 음식, 문화관광, 쇼핑, 숙박
* 전남 : 음식, 자연관광, 문화관광, 숙박, 레저스포츠
* 전북 : 음식, 문화관광, 자연관광, 레저스포츠, 숙박
* 제주 : 자연관광, 음식, 문화관광, 숙박, 레저스포츠
* 충남 : 자연관광, 문화관광, 음식, 쇼핑, 레저스포츠
* 충북 : 레저스포츠, 음식, 자연관광, 문화관광, 숙박


In [39]:
for i in list_concept_region.keys():
    print("{} : {}".format(i,list_concept_region[i]))

레저스포츠 : ['충북']
문화관광 : ['강원', '경기', '경남', '경북', '광주', '대구', '대전', '부산', '서울', '세종', '울산', '인천', '전남', '전북', '제주', '충남']
쇼핑 : ['경기', '광주', '대구', '대전', '부산', '서울']
숙박 : []
음식 : ['강원', '경기', '경남', '경북', '광주', '대구', '대전', '서울', '세종', '울산', '인천', '전남', '전북', '제주', '충남', '충북']
자연관광 : ['강원', '경남', '경북', '부산', '세종', '울산', '인천', '전남', '전북', '제주', '충남', '충북']


### 목적지 유형별 검색량이 많은 월
* 레저스포츠 : 10월, 5월, 6월
* 문화관광 : 10월, 5월, 8월
* 쇼핑 : 10월, 12월, 9월
* 숙박 : 8월, 10월, 7월
* 음식 : 8월, 7월, 10월
* 자연관광 : 8월, 10월, 9월


In [40]:
pivot_concept=pd.pivot_table(data=df_concept,index="destination_type",columns = "std_month",values='destination_search',aggfunc='sum')
# pivot_concept.loc['숙박',:].sort_values(ascending=True)[:3]
pivot_concept

std_month,1,2,3,4,5,6,7,8,9,10,11,12
destination_type,,,,,,,,,,,,
레저스포츠,3700478,4585856,6461137,7988881,8896855,8789573,8335529,8405937,8564853,9839209,7676060,4626987
문화관광,14624554,14730420,16360806,20643126,23527229,20665777,20067249,22866697,21508205,28707042,20090142,16264725
쇼핑,17019729,15656896,16485797,17447824,19555274,17979481,18579223,19193786,19678210,21860562,19396057,20139183
숙박,10695561,10852072,9698585,12367927,14572387,14873705,16979050,19402626,14000229,17221704,14162680,13840679
음식,39818247,41476147,44438930,50879117,59404086,56639022,61723165,64934420,53996243,61685220,52159177,51746719
자연관광,5333026,5467300,6031767,6674182,7883466,7631843,8593100,11182396,8845864,10357972,6368949,5184728


In [41]:
for i in list(pivot_concept.index):
               print("* {} : {}월, {}월, {}월".format(i,pivot_concept.loc[i].sort_values(ascending=False).keys()[0],pivot_concept.loc[i].sort_values(ascending=False).keys()[1],pivot_concept.loc[i].sort_values(ascending=False).keys()[2]))

* 레저스포츠 : 10월, 5월, 6월
* 문화관광 : 10월, 5월, 8월
* 쇼핑 : 10월, 12월, 9월
* 숙박 : 8월, 10월, 7월
* 음식 : 8월, 7월, 10월
* 자연관광 : 8월, 10월, 9월


### 키워드별 다른 달에 비해 검색량이 많은 달
* 레저스포츠 : 6월, 4월, 9월
* 문화관광 : 10월, 4월, 5월
* 쇼핑 : 1월, 12월, 2월
* 숙박 : 8월, 7월, 12월
* 음식 : 12월, 7월, 6월
* 자연관광 : 8월, 9월, 10월

In [94]:
pivot_concept=pd.pivot_table(data=df_concept,index="std_month",columns = "destination_type",values='destination_search',aggfunc='sum')
pivot_concept_month_ratio = pivot_concept.copy()
for j in list(pivot_concept.columns):
    for i in range(1,13):
        pivot_concept_month_ratio.loc[i, j] = float(pivot_concept.loc[i, j] / pivot_concept.loc[i, :].sum())
pivot_concept_month_ratio

destination_type,레저스포츠,문화관광,쇼핑,숙박,음식,자연관광
std_month,,,,,,
1,0.040579,0.160372,0.186637,0.117287,0.436644,0.058482
2,0.049433,0.158787,0.168773,0.116980,0.447092,0.058935
3,0.064951,0.164468,0.165725,0.097496,0.446726,0.060635
4,0.068869,0.177956,0.150411,0.106619,0.438609,0.057536
5,0.066474,0.175787,0.146110,0.108880,0.443846,0.058902
6,0.069439,0.163263,0.142041,0.117505,0.447458,0.060293
7,0.062077,0.149446,0.138365,0.126448,0.459669,0.063995
8,0.057580,0.156636,0.131477,0.132908,0.444799,0.076599
9,0.067656,0.169900,0.155444,0.110592,0.426532,0.069876


In [95]:
for i in list(pivot_concept_month_ratio.columns):
               print("* {} : {}월, {}월, {}월".format(i,pivot_concept_month_ratio[i].sort_values(ascending=False).keys()[0],pivot_concept_month_ratio[i].sort_values(ascending=False).keys()[1],pivot_concept_month_ratio[i].sort_values(ascending=False).keys()[2]))

* 레저스포츠 : 6월, 4월, 9월
* 문화관광 : 10월, 4월, 5월
* 쇼핑 : 1월, 12월, 2월
* 숙박 : 8월, 7월, 12월
* 음식 : 12월, 7월, 6월
* 자연관광 : 8월, 9월, 10월


### 월별 다른 달에 비해 검색량이 많은 목적지 유형
* 1월 : 쇼핑, 숙박, 음식
* 2월 : 쇼핑, 음식, 숙박
* 3월 : 쇼핑, 레저스포츠, 음식
* 4월 : 레저스포츠, 문화관광, 음식
* 5월 : 레저스포츠, 문화관광, 음식
* 6월 : 레저스포츠, 음식, 숙박
* 7월 : 숙박, 음식, 자연관광
* 8월 : 자연관광, 숙박, 음식
* 9월 : 자연관광, 레저스포츠, 문화관광
* 10월 : 문화관광, 자연관광, 레저스포츠
* 11월 : 레저스포츠, 쇼핑, 숙박
* 12월 : 쇼핑, 숙박, 음식

In [96]:
pivot_concept=pd.pivot_table(data=df_concept,index="std_month",columns = "destination_type",values='destination_search',aggfunc='sum')
pivot_concept_keyword_ratio = pivot_concept.copy()
for j in list(pivot_concept.columns):
    for i in range(1,13):
        pivot_concept_keyword_ratio.loc[i, j] = float(pivot_concept.loc[i, j] / pivot_concept.loc[:, j].sum())
pivot_concept_keyword_ratio

destination_type,레저스포츠,문화관광,쇼핑,숙박,음식,자연관광
std_month,,,,,,
1,0.042112,0.060921,0.076324,0.063412,0.062323,0.059551
2,0.052188,0.061362,0.070213,0.064340,0.064918,0.061050
3,0.073530,0.068154,0.073930,0.057501,0.069555,0.067353
4,0.090916,0.085993,0.078244,0.073327,0.079635,0.074526
5,0.101249,0.098007,0.087695,0.086397,0.092979,0.088030
6,0.100028,0.086087,0.080628,0.088184,0.088651,0.085220
7,0.094861,0.083594,0.083318,0.100666,0.096608,0.095954
8,0.095662,0.095256,0.086074,0.115035,0.101635,0.124867
9,0.097470,0.089597,0.088246,0.083005,0.084514,0.098776


In [43]:
for i in list(pivot_concept_ratio.index):
               print("* {}월 : {}, {}, {}".format(i,pivot_concept_ratio.loc[i].sort_values(ascending=False).keys()[0],pivot_concept_ratio.loc[i].sort_values(ascending=False).keys()[1],pivot_concept_ratio.loc[i].sort_values(ascending=False).keys()[2]))

* 1월 : 쇼핑, 숙박, 음식
* 2월 : 쇼핑, 음식, 숙박
* 3월 : 쇼핑, 레저스포츠, 음식
* 4월 : 레저스포츠, 문화관광, 음식
* 5월 : 레저스포츠, 문화관광, 음식
* 6월 : 레저스포츠, 음식, 숙박
* 7월 : 숙박, 음식, 자연관광
* 8월 : 자연관광, 숙박, 음식
* 9월 : 자연관광, 레저스포츠, 문화관광
* 10월 : 문화관광, 자연관광, 레저스포츠
* 11월 : 레저스포츠, 쇼핑, 숙박
* 12월 : 쇼핑, 숙박, 음식


### 지역별 관광지 검색량이 많은 월
* 강원 : 8월, 7월, 10월
* 경기 : 10월, 5월, 8월
* 경남 : 8월, 10월, 5월
* 경북 : 8월, 10월, 7월
* 광주 : 10월, 5월, 8월
* 대구 : 10월, 8월, 5월
* 대전 : 10월, 5월, 11월
* 부산 : 8월, 10월, 7월
* 서울 : 10월, 12월, 5월
* 세종 : 10월, 5월, 8월
* 울산 : 10월, 8월, 7월
* 인천 : 10월, 9월, 8월
* 전남 : 8월, 10월, 5월
* 전북 : 10월, 8월, 5월
* 제주 : 8월, 10월, 7월
* 충남 : 10월, 8월, 5월
* 충북 : 10월, 8월, 5월

In [44]:
pivot_visitor=pd.pivot_table(data=df_visitor,index="region",columns = "std_month",values='destination_search',aggfunc='sum')
pivot_visitor

std_month,1,2,3,4,5,6,7,8,9,10,11,12
region,,,,,,,,,,,,
강원,7767478,8184410,6266780,8407164,10827689,11275479,13504128,16922400,10449981,13401478,8629417,8382086
경기,29037338,29866345,34700950,39838959,45064630,42110927,43389952,43544232,42730581,49105154,39629855,36449990
경남,4775395,4788539,5276199,6108008,7292774,6543413,7080676,8576067,6422574,8523524,6383504,5599623
경북,4412270,4448026,4308810,5515679,6439660,5902943,6566697,8287001,6161474,8248791,6224431,5049459
광주,1500557,1442671,1604907,1818671,2101026,2039403,1994542,2065924,1955073,2211906,2006505,1931611
대구,2922476,2773642,2978242,3414551,3909276,3649713,3898437,3989658,3839255,4399470,3893262,3862327
대전,2183146,2158621,2290398,2586650,2970165,2841775,2858808,2934732,2902778,3288571,2949171,2867212
부산,5891505,5650624,5651545,6290199,7439319,7052593,7523495,8074894,6795579,7835391,6829107,6981931
서울,17472058,16875737,17890639,19302764,21632022,20599907,21571987,21392555,20638644,22630650,20908941,21658818


In [45]:
for i in list(pivot_visitor.index):
               print("{} : {}월, {}월, {}월".format(i,pivot_visitor.loc[i].sort_values(ascending=False).keys()[0],pivot_visitor.loc[i].sort_values(ascending=False).keys()[1],pivot_visitor.loc[i].sort_values(ascending=False).keys()[2]))

강원 : 8월, 7월, 10월
경기 : 10월, 5월, 8월
경남 : 8월, 10월, 5월
경북 : 8월, 10월, 7월
광주 : 10월, 5월, 8월
대구 : 10월, 8월, 5월
대전 : 10월, 5월, 11월
부산 : 8월, 10월, 7월
서울 : 10월, 12월, 5월
세종 : 10월, 5월, 8월
울산 : 10월, 8월, 7월
인천 : 10월, 9월, 8월
전남 : 8월, 10월, 5월
전북 : 10월, 8월, 5월
제주 : 8월, 10월, 7월
충남 : 10월, 8월, 5월
충북 : 10월, 8월, 5월


### 월별 다른 월에 비해 방문지 검색량이 많은 지역
* 1월 : 서울, 부산, 인천
* 2월 : 서울, 제주, 부산
* 3월 : 서울, 세종, 경기
* 4월 : 충남, 경기, 세종
* 5월 : 전북, 충북, 충남
* 6월 : 충북, 강원, 제주
* 7월 : 강원, 충북, 제주
* 8월 : 강원, 전남, 경북
* 9월 : 충남, 인천, 충북
* 10월 : 충북, 경북, 전북
* 11월 : 충북, 대전, 대구
* 12월 : 서울, 대구, 대전

In [64]:
pivot_visitor_month=pd.pivot_table(data=df_visitor,index="std_month",columns = "region",values='destination_search',aggfunc='sum')
pivot_visitor_month_ratio = pivot_visitor_month.copy()
for j in list(pivot_visitor_month.columns):
    for i in range(1,13):
        pivot_visitor_month_ratio.loc[i, j] = float(pivot_visitor_month.loc[i, j] / pivot_visitor_month.loc[:, j].sum())
pivot_visitor_month_ratio

region,강원,경기,경남,경북,광주,대구,대전,부산,서울,세종,울산,인천,전남,전북,제주,충남,충북
std_month,,,,,,,,,,,,,,,,,
1,0.062632,0.061071,0.061721,0.061654,0.066183,0.067137,0.066494,0.071833,0.072028,0.062186,0.067419,0.068590,0.059382,0.061079,0.066103,0.056501,0.051027
2,0.065993,0.062815,0.061891,0.062153,0.063630,0.063717,0.065747,0.068896,0.069569,0.065239,0.065679,0.066140,0.060776,0.061537,0.069010,0.059169,0.055467
3,0.050531,0.072983,0.068194,0.060208,0.070786,0.068418,0.069761,0.068908,0.073753,0.073719,0.068173,0.071030,0.067325,0.062802,0.069165,0.066484,0.064513
4,0.067790,0.083789,0.078945,0.077072,0.080214,0.078441,0.078784,0.076695,0.079575,0.083095,0.074617,0.078566,0.082374,0.080541,0.079656,0.086792,0.083089
5,0.087307,0.094779,0.094258,0.089983,0.092667,0.089806,0.090465,0.090706,0.089177,0.092259,0.090261,0.090013,0.094855,0.095449,0.090370,0.095246,0.095313
6,0.090918,0.088567,0.084573,0.082483,0.089949,0.083843,0.086555,0.085990,0.084922,0.086659,0.084248,0.084538,0.083738,0.085401,0.090042,0.088893,0.090959
7,0.108888,0.091257,0.091517,0.091758,0.087971,0.089557,0.087074,0.091732,0.088929,0.088591,0.091502,0.087412,0.089365,0.088056,0.092797,0.090021,0.093969
8,0.136451,0.091582,0.110844,0.115796,0.091119,0.091652,0.089386,0.098455,0.088190,0.090429,0.102295,0.093179,0.118114,0.106562,0.111491,0.099126,0.101140
9,0.084261,0.089870,0.083011,0.086096,0.086230,0.088197,0.088413,0.082857,0.085082,0.088353,0.083272,0.094309,0.085700,0.090140,0.079002,0.094550,0.090975


In [66]:
for i in list(pivot_visitor_month_ratio.index):
               print("* {}월 : {}, {}, {}".format(i,pivot_visitor_month_ratio.loc[i].sort_values(ascending=False).keys()[0],pivot_visitor_month_ratio.loc[i].sort_values(ascending=False).keys()[1],pivot_visitor_month_ratio.loc[i].sort_values(ascending=False).keys()[2]))

* 1월 : 서울, 부산, 인천
* 2월 : 서울, 제주, 부산
* 3월 : 서울, 세종, 경기
* 4월 : 충남, 경기, 세종
* 5월 : 전북, 충북, 충남
* 6월 : 충북, 강원, 제주
* 7월 : 강원, 충북, 제주
* 8월 : 강원, 전남, 경북
* 9월 : 충남, 인천, 충북
* 10월 : 충북, 경북, 전북
* 11월 : 충북, 대전, 대구
* 12월 : 서울, 대구, 대전


### 지역별 다른 월에 비해 방문지 검색량이 많은 달
* 강원 : 8월, 7월, 6월
* 경기 : 3월, 4월, 5월
* 경남 : 8월, 10월, 5월
* 경북 : 8월, 10월, 11월
* 광주 : 12월, 11월, 1월
* 대구 : 12월, 11월, 1월
* 대전 : 12월, 11월, 1월
* 부산 : 1월, 12월, 2월
* 서울 : 12월, 1월, 2월
* 세종 : 3월, 12월, 4월
* 울산 : 12월, 1월, 11월
* 인천 : 9월, 1월, 12월
* 전남 : 8월, 10월, 5월
* 전북 : 10월, 8월, 11월
* 제주 : 8월, 2월, 6월
* 충남 : 10월, 4월, 9월
* 충북 : 10월, 11월, 6월

In [82]:
pivot_visitor_region=pd.pivot_table(data=df_visitor,index="std_month",columns = "region",values='destination_search',aggfunc='sum')
pivot_visitor_region_ratio = pivot_visitor_region.copy()
for j in list(pivot_visitor_region.columns):
    for i in range(1,13):
        pivot_visitor_region_ratio.loc[i, j] = float(pivot_visitor_region.loc[i, j] / pivot_visitor_region.loc[i, :].sum())
pivot_visitor_region_ratio

region,강원,경기,경남,경북,광주,대구,대전,부산,서울,세종,울산,인천,전남,전북,제주,충남,충북
std_month,,,,,,,,,,,,,,,,,
1,0.075633,0.282740,0.046499,0.042963,0.014611,0.028457,0.021258,0.057366,0.170128,0.006231,0.014425,0.068338,0.031897,0.026161,0.044228,0.044128,0.024939
2,0.079151,0.288836,0.046310,0.043017,0.013952,0.026824,0.020876,0.054647,0.163205,0.006492,0.013958,0.065449,0.032425,0.026178,0.045859,0.045898,0.026925
3,0.056819,0.314625,0.047838,0.039067,0.014551,0.027003,0.020767,0.051241,0.162210,0.006878,0.013583,0.065896,0.033675,0.025047,0.043090,0.048350,0.029360
4,0.065421,0.310009,0.047530,0.042921,0.014152,0.026571,0.020128,0.048948,0.150206,0.006654,0.012759,0.062556,0.035361,0.027569,0.042592,0.054171,0.032454
5,0.073217,0.304729,0.049314,0.043545,0.014207,0.026435,0.020084,0.050305,0.146277,0.006419,0.013412,0.062280,0.035384,0.028391,0.041990,0.051659,0.032351
6,0.080731,0.301507,0.046850,0.042264,0.014602,0.026131,0.020347,0.050495,0.147492,0.006384,0.013255,0.061933,0.033075,0.026896,0.044299,0.051050,0.032689
7,0.091633,0.294425,0.048046,0.044559,0.013534,0.026453,0.019399,0.051051,0.146378,0.006186,0.013644,0.060691,0.033452,0.026283,0.043268,0.048995,0.032005
8,0.105507,0.271487,0.053469,0.051667,0.012880,0.024874,0.018297,0.050345,0.133377,0.005801,0.014015,0.059443,0.040625,0.029225,0.047764,0.049572,0.031652
9,0.074347,0.304010,0.045694,0.043836,0.013910,0.027315,0.020652,0.048348,0.146835,0.006468,0.013019,0.068655,0.033636,0.028209,0.038622,0.053956,0.032488


In [74]:
for i in list(pivot_visitor_region_ratio.columns):
               print("* {} : {}월, {}월, {}월".format(i,pivot_visitor_region_ratio[i].sort_values(ascending=False).keys()[0],pivot_visitor_region_ratio[i].sort_values(ascending=False).keys()[1],pivot_visitor_region_ratio[i].sort_values(ascending=False).keys()[2]))

* 강원 : 8월, 7월, 6월
* 경기 : 3월, 4월, 5월
* 경남 : 8월, 10월, 5월
* 경북 : 8월, 10월, 11월
* 광주 : 12월, 11월, 1월
* 대구 : 12월, 11월, 1월
* 대전 : 12월, 11월, 1월
* 부산 : 1월, 12월, 2월
* 서울 : 12월, 1월, 2월
* 세종 : 3월, 12월, 4월
* 울산 : 12월, 1월, 11월
* 인천 : 9월, 1월, 12월
* 전남 : 8월, 10월, 5월
* 전북 : 10월, 8월, 11월
* 제주 : 8월, 2월, 6월
* 충남 : 10월, 4월, 9월
* 충북 : 10월, 11월, 6월


## CDA

### 월별 전국 대비 지역 방문자 차지 비율 정규분포 확인
#### 정규 분포 : 강원 경기 경남 경북 광주 대구 대전 부산 서울 세종 울산 인천 전남 전북 제주 충남 충북 
#### 비정규 분포 : 

In [83]:
# pvalue > 0.05 : 정규분포
in_list = "정규 분포 : "
not_list = "비정규 분포 : "
for i in list(pivot_visitor_region_ratio.columns):
    if stats.shapiro(pivot_visitor_region_ratio[i].values)[1] > 0.05:
        print("* {} : 정규분포".format(i))
        print("  * p-value = {}".format(stats.shapiro(pivot_visitor_region_ratio[i].values)[1]))
        in_list = in_list + i + " "
    else:
        print("* {} : 비정규분포".format(i))
        print("  * p-value = {}".format(stats.shapiro(pivot_visitor_region_ratio[i].values)[1]))
        not_list = not_list + i + " "
print("")
print(in_list)
print("")
print(not_list)
        # list(pivot_visitor.index)

* 강원 : 정규분포
  * p-value = 0.5518659479574947
* 경기 : 정규분포
  * p-value = 0.8721341610062086
* 경남 : 정규분포
  * p-value = 0.10915853801037734
* 경북 : 정규분포
  * p-value = 0.2296358694005316
* 광주 : 정규분포
  * p-value = 0.998795916990159
* 대구 : 정규분포
  * p-value = 0.07502492975417027
* 대전 : 정규분포
  * p-value = 0.9445908508980316
* 부산 : 정규분포
  * p-value = 0.16172961656221208
* 서울 : 정규분포
  * p-value = 0.6594827732803284
* 세종 : 정규분포
  * p-value = 0.8048146627116481
* 울산 : 정규분포
  * p-value = 0.9793651383130646
* 인천 : 정규분포
  * p-value = 0.3976248914861544
* 전남 : 정규분포
  * p-value = 0.38606687809382245
* 전북 : 정규분포
  * p-value = 0.956698232372812
* 제주 : 정규분포
  * p-value = 0.6299063730276232
* 충남 : 정규분포
  * p-value = 0.40986789886047137
* 충북 : 정규분포
  * p-value = 0.13844897402727013

정규 분포 : 강원 경기 경남 경북 광주 대구 대전 부산 서울 세종 울산 인천 전남 전북 제주 충남 충북 

비정규 분포 : 


### 월별 전체 키워드 대비 각각의 키워드 차지 비율 정규분포 확인
#### 정규 분포 : 문화관광 쇼핑 숙박 음식 자연관광 

#### 비정규 분포 : 레저스포츠 

In [97]:
# pvalue > 0.05 : 정규분포
in_list = "정규 분포 : "
not_list = "비정규 분포 : "
for i in list(pivot_concept_month_ratio.columns):
    if stats.shapiro(pivot_concept_month_ratio[i].values)[1] > 0.05:
        print("* {} : 정규분포".format(i))
        print("  * p-value = {}".format(stats.shapiro(pivot_concept_month_ratio[i].values)[1]))
        in_list = in_list + i + " "

    else:
        print("* {} : 비정규분포".format(i))
        print("  * p-value = {}".format(stats.shapiro(pivot_concept_month_ratio[i].values)[1]))
        not_list = not_list + i + " "
print("")
print(in_list)
print("")
print(not_list)

* 레저스포츠 : 비정규분포
  * p-value = 0.011868600301556172
* 문화관광 : 정규분포
  * p-value = 0.9662754332464113
* 쇼핑 : 정규분포
  * p-value = 0.8384519684852966
* 숙박 : 정규분포
  * p-value = 0.978254304822841
* 음식 : 정규분포
  * p-value = 0.5941700620452111
* 자연관광 : 정규분포
  * p-value = 0.7594278273911488

정규 분포 : 문화관광 쇼핑 숙박 음식 자연관광 

비정규 분포 : 레저스포츠 


### 레저스포츠 : 충북, 경기, 강원, 인천, 경남
#### 상관관계 있음 : 경기 충북 

#### 상관관계 없음 : 강원 경남 인천 

#### 상관계수
* 충북 : 0.87
* 경기 : 0.59


In [106]:
# pvalue > 0.05 : 상관관계가 없음
in_list = "상관관계 있음 : "
not_list = "상관관계 없음 : "
pivot_visitor_region_ratio.columns
df_correlation = pd.DataFrame(data=pivot_visitor_region_ratio.columns,columns = ['region'])
for i in pivot_visitor_month_ratio.columns :
    if stats.spearmanr(pivot_visitor_region_ratio[i].values,pivot_concept_month_ratio['레저스포츠'].values)[1] < 0.05:
        df_correlation.loc[df_correlation.query(f"region == '{i}'").index,'p-value'] = round(stats.spearmanr(pivot_visitor_region_ratio[i].values,pivot_concept_month_ratio['레저스포츠'].values)[1],5)
        df_correlation.loc[df_correlation.query(f"region == '{i}'").index,'correlation'] = round(pd.DataFrame(pivot_visitor_region_ratio[i].values,pivot_concept_month_ratio['레저스포츠'].values).reset_index().corr().loc['index',0],2)
        in_list = in_list + i + " "

    else:
        df_correlation.loc[df_correlation.query(f"region == '{i}'").index,'p-value'] = round(stats.spearmanr(pivot_visitor_region_ratio[i].values,pivot_concept_month_ratio['레저스포츠'].values)[1],5)
        not_list = not_list + i + " "
print("")
print(in_list)
print("")
print(not_list)
df_correlation.sort_values(by="correlation", ascending=False)


상관관계 있음 : 경기 부산 울산 충남 충북 

상관관계 없음 : 강원 경남 경북 광주 대구 대전 서울 세종 인천 전남 전북 제주 


,region,p-value,correlation
16,충북,0.00451,0.87
15,충남,0.00114,0.87
1,경기,0.00259,0.59
10,울산,0.00008,-0.82
7,부산,0.00451,-0.92
0,강원,0.72919,NaN
2,경남,0.61752,NaN
3,경북,0.84593,NaN
4,광주,0.60210,NaN
5,대구,0.16670,NaN


### 문화 관광 : 경기, 서울, 강원, 경북, 인천
#### 상관관계 있음 :

#### 상관관계 없음 : 강원 경기 경북 서울 인천 

#### 상관계수

In [105]:
# pvalue > 0.05 : 상관관계가 없음
in_list = "상관관계 있음 : "
not_list = "상관관계 없음 : "
pivot_visitor_month_ratio.columns
df_correlation = pd.DataFrame(data=pivot_visitor_region_ratio.columns,columns = ['region'])
for i in pivot_visitor_month_ratio.columns :
    if stats.pearsonr(pivot_visitor_region_ratio[i].values,pivot_concept_month_ratio['문화관광'].values)[1] < 0.05:
        df_correlation.loc[df_correlation.query(f"region == '{i}'").index,'p-value'] = round(stats.pearsonr(pivot_visitor_region_ratio[i].values,pivot_concept_month_ratio['문화관광'].values)[1],5)
        df_correlation.loc[df_correlation.query(f"region == '{i}'").index,'correlation'] = round(pd.DataFrame(pivot_visitor_region_ratio[i].values,pivot_concept_month_ratio['문화관광'].values).reset_index().corr().loc['index',0],2)
        in_list = in_list + i + " "

    else:
        df_correlation.loc[df_correlation.query(f"region == '{i}'").index,'p-value'] = round(stats.pearsonr(pivot_visitor_region_ratio[i].values,pivot_concept_month_ratio['문화관광'].values)[1],5)
        not_list = not_list + i + " "
print("")
print(in_list)
print("")
print(not_list)
df_correlation.sort_values(by="correlation", ascending=False)


상관관계 있음 : 부산 전북 충남 충북 

상관관계 없음 : 강원 경기 경남 경북 광주 대구 대전 서울 세종 울산 인천 전남 제주 


,region,p-value,correlation
15,충남,0.00386,0.76
13,전북,0.01180,0.70
16,충북,0.03158,0.62
7,부산,0.01261,-0.69
0,강원,0.46160,NaN
1,경기,0.22143,NaN
2,경남,0.29779,NaN
3,경북,0.36666,NaN
4,광주,0.34526,NaN
5,대구,0.27116,NaN


### 쇼핑 : 경기, 서울, 부산, 인천, 대전
#### 상관관계 있음 : 대전 부산 서울 인천

#### 상관관계 없음 : 경기 

#### 상관계수
* 서울 : 0.93
* 인천 : 0.84
* 대전 : 0.83
* 부산 : 0.78


In [104]:
# pvalue > 0.05 : 상관관계가 없음
in_list = "상관관계 있음 : "
not_list = "상관관계 없음 : "
pivot_visitor_month_ratio.columns
df_correlation = pd.DataFrame(data=pivot_visitor_region_ratio.columns,columns = ['region'])
for i in pivot_visitor_month_ratio.columns :
    if stats.pearsonr(pivot_visitor_region_ratio[i].values,pivot_concept_month_ratio['쇼핑'].values)[1] < 0.05:
        df_correlation.loc[df_correlation.query(f"region == '{i}'").index,'p-value'] = round(stats.pearsonr(pivot_visitor_region_ratio[i].values,pivot_concept_month_ratio['쇼핑'].values)[1],5)
        df_correlation.loc[df_correlation.query(f"region == '{i}'").index,'correlation'] = round(pd.DataFrame(pivot_visitor_region_ratio[i].values,pivot_concept_month_ratio['쇼핑'].values).reset_index().corr().loc['index',0],2)
        in_list = in_list + i + " "

    else:
        df_correlation.loc[df_correlation.query(f"region == '{i}'").index,'p-value'] = round(stats.pearsonr(pivot_visitor_region_ratio[i].values,pivot_concept_month_ratio['쇼핑'].values)[1],5)
        not_list = not_list + i + " "
print("")
print(in_list)
print("")
print(not_list)
df_correlation.sort_values(by="correlation", ascending=False)


상관관계 있음 : 강원 경남 광주 대구 대전 부산 서울 인천 전남 전북 충남 충북 

상관관계 없음 : 경기 경북 세종 울산 제주 


,region,p-value,correlation
8,서울,0.00001,0.93
11,인천,0.00065,0.84
6,대전,0.00085,0.83
5,대구,0.00220,0.79
7,부산,0.00287,0.78
4,광주,0.00986,0.71
0,강원,0.03972,-0.60
13,전북,0.03006,-0.62
2,경남,0.01585,-0.68
15,충남,0.01345,-0.69


###  숙박 : 강원, 서울, 제주, 부산, 인천
#### 상관관계 있음 : 강원 

#### 상관관계 없음 : 부산 서울 인천 제주 

#### 상관계수
* 강원 : 0.79

In [107]:
# pvalue > 0.05 : 상관관계가 없음
in_list = "상관관계 있음 : "
not_list = "상관관계 없음 : "
pivot_visitor_month_ratio.columns
df_correlation = pd.DataFrame(data=pivot_visitor_region_ratio.columns,columns = ['region'])
for i in pivot_visitor_month_ratio.columns :
    if stats.pearsonr(pivot_visitor_region_ratio[i].values,pivot_concept_month_ratio['숙박'].values)[1] < 0.05:
        df_correlation.loc[df_correlation.query(f"region == '{i}'").index,'p-value'] = round(stats.pearsonr(pivot_visitor_region_ratio[i].values,pivot_concept_month_ratio['숙박'].values)[1],5)
        df_correlation.loc[df_correlation.query(f"region == '{i}'").index,'correlation'] = round(pd.DataFrame(pivot_visitor_region_ratio[i].values,pivot_concept_month_ratio['숙박'].values).reset_index().corr().loc['index',0],2)
        in_list = in_list + i + " "

    else:
        df_correlation.loc[df_correlation.query(f"region == '{i}'").index,'p-value'] = round(stats.pearsonr(pivot_visitor_region_ratio[i].values,pivot_concept_month_ratio['숙박'].values)[1],5)
        not_list = not_list + i + " "
print("")
print(in_list)
print("")
print(not_list)
df_correlation.sort_values(by="correlation", ascending=False)


상관관계 있음 : 강원 경기 경북 세종 

상관관계 없음 : 경남 광주 대구 대전 부산 서울 울산 인천 전남 전북 제주 충남 충북 


,region,p-value,correlation
0,강원,0.00235,0.79
3,경북,0.04900,0.58
9,세종,0.00801,-0.72
1,경기,0.00052,-0.85
2,경남,0.34863,NaN
4,광주,0.43446,NaN
5,대구,0.95933,NaN
6,대전,0.59386,NaN
7,부산,0.43916,NaN
8,서울,0.56435,NaN


###  음식 : 경기, 강원, 서울, 인천, 제주
#### 상관관계 있음 : 

#### 상관관계 없음 : 강원 경기 서울 인천 제주 

#### 상관계수

In [108]:
# pvalue > 0.05 : 상관관계가 없음
in_list = "상관관계 있음 : "
not_list = "상관관계 없음 : "
pivot_visitor_month_ratio.columns
df_correlation = pd.DataFrame(data=pivot_visitor_region_ratio.columns,columns = ['region'])
for i in pivot_visitor_month_ratio.columns :
    if stats.pearsonr(pivot_visitor_region_ratio[i].values,pivot_concept_month_ratio['음식'].values)[1] < 0.05:
        df_correlation.loc[df_correlation.query(f"region == '{i}'").index,'p-value'] = round(stats.pearsonr(pivot_visitor_region_ratio[i].values,pivot_concept_month_ratio['음식'].values)[1],5)
        df_correlation.loc[df_correlation.query(f"region == '{i}'").index,'correlation'] = round(pd.DataFrame(pivot_visitor_region_ratio[i].values,pivot_concept_month_ratio['음식'].values).reset_index().corr().loc['index',0],2)
        in_list = in_list + i + " "

    else:
        df_correlation.loc[df_correlation.query(f"region == '{i}'").index,'p-value'] = round(stats.pearsonr(pivot_visitor_region_ratio[i].values,pivot_concept_month_ratio['음식'].values)[1],5)
        not_list = not_list + i + " "
print("")
print(in_list)
print("")
print(not_list)
df_correlation.sort_values(by="correlation", ascending=False)


상관관계 있음 : 전북 충남 

상관관계 없음 : 강원 경기 경남 경북 광주 대구 대전 부산 서울 세종 울산 인천 전남 제주 충북 


,region,p-value,correlation
15,충남,0.04162,-0.59
13,전북,0.00633,-0.74
0,강원,0.82480,NaN
1,경기,0.85968,NaN
2,경남,0.36023,NaN
3,경북,0.13043,NaN
4,광주,0.25146,NaN
5,대구,0.47049,NaN
6,대전,0.52719,NaN
7,부산,0.06443,NaN


### 자연관광 : 경기, 강원, 인천, 제주, 충남
#### 상관관계 있음 : 강원 

#### 상관관계 없음 : 경기 인천 제주 충남

#### 상관계수
* 강원 : 0.68

In [109]:
# pvalue > 0.05 : 상관관계가 없음
in_list = "상관관계 있음 : "
not_list = "상관관계 없음 : "
pivot_visitor_month_ratio.columns
df_correlation = pd.DataFrame(data=pivot_visitor_region_ratio.columns,columns = ['region'])
for i in pivot_visitor_month_ratio.columns :
    if stats.pearsonr(pivot_visitor_region_ratio[i].values,pivot_concept_month_ratio['자연관광'].values)[1] < 0.05:
        df_correlation.loc[df_correlation.query(f"region == '{i}'").index,'p-value'] = round(stats.pearsonr(pivot_visitor_region_ratio[i].values,pivot_concept_month_ratio['자연관광'].values)[1],5)
        df_correlation.loc[df_correlation.query(f"region == '{i}'").index,'correlation'] = round(pd.DataFrame(pivot_visitor_region_ratio[i].values,pivot_concept_month_ratio['자연관광'].values).reset_index().corr().loc['index',0],2)
        in_list = in_list + i + " "

    else:
        df_correlation.loc[df_correlation.query(f"region == '{i}'").index,'p-value'] = round(stats.pearsonr(pivot_visitor_region_ratio[i].values,pivot_concept_month_ratio['자연관광'].values)[1],5)
        not_list = not_list + i + " "
print("")
print(in_list)
print("")
print(not_list)
df_correlation.sort_values(by="correlation", ascending=False)


상관관계 있음 : 강원 경남 경북 광주 대구 대전 부산 서울 세종 전남 전북 

상관관계 없음 : 경기 울산 인천 제주 충남 충북 


,region,p-value,correlation
12,전남,0.00219,0.79
2,경남,0.01524,0.68
0,강원,0.01411,0.68
3,경북,0.01952,0.66
13,전북,0.02925,0.63
7,부산,0.04693,-0.58
9,세종,0.01467,-0.68
5,대구,0.00208,-0.79
8,서울,0.00163,-0.80
6,대전,0.00036,-0.86


## 평가

### 분석 과정
1. 데이터 정제
월별 전체 키워드 대비 각각의 키워드 차지 비율 계산
월별 전국 관광지 방문객 대비 해당 지역 방문객 비율 계산.
지역별 키워드 관련 관광지 방문객이 많은 키워드들을 계산.

2. 데이터 비교
키워드 관련 관광지 방문객이 많은 지역과 해당 지역의 월별 차지 비율 변화량을 비교

### 분석 결과
#### 레저 스포츠
* 충북, 경기, 강원, 인천, 경남 중 경기 충북만 상관관계가 있음
* 충북 : 0.87
* 경기 : 0.59
#### 문화 관광
* 경기, 서울, 강원, 경북, 인천 중 상관관계가 있는 지역이 없음

#### 쇼핑
* 경기, 서울, 부산, 인천, 대전 중 대전, 부산, 서울, 인천이 상관관계가 있음
* 서울 : 0.93
* 인천 : 0.84
* 대전 : 0.83
* 부산 : 0.78
#### 숙박
* 강원, 서울, 제주, 부산, 인천 중 강원만 상관관계가 있음
* 강원 : 0.79
#### 음식
* 경기, 강원, 서울, 인천, 제주 중 상관관계가 있는 지역이 없음

#### 자연관광
* 경기 강원, 인천, 제주, 충남 중 강원만 상관관계가 있음 
* 강원 : 0.68

### 결론
* 쇼핑을 제외하고는 키워드의 검색량과 키워드와 관련된 관광지를 많이 방문한 지역 전체 방문객 수의 상관관계를 찾기 어려움
* 그렇기 때문에 여행 컨셉의 검색량을 바탕으로 여행지를 추천해주기에는 무리가 있음
